In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

## 1. Forward Imaging Model

In this process, the sample is illuminated with one LED of the LED illuminator at time and a camera captures Lo-Res images under different incident angles. To simulate it, we have to:

1. Create a Hi-Res complex image.

2. Generate the incident wave vectors.

3. Produce the ouput Lo-Res images.

#####  [2. The Recovery Process](phaseretrieval.ipynb)

In [ ]:
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
import decimal
import scipy.misc
import os

In [ ]:
%matplotlib inline

#### 1.1. Create the Hi-Res complex image.

In [ ]:
def generate_obj(amplitude, phase, **kwargs):
    """Returns the Hi-Res complex object ('numpy.ndarray'),
    the sample."""
    # Show the output image in the notebook.
    show = kwargs.pop('show', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    amplitude = Image.open(amplitude)
    phase = Image.open(phase)
    w, h = amplitude.size  # It is not the same that
                           # numpy.ndarray.shape
    # Some programming languages such as MATLAB, the resize methods
    # use bicubic interpolation by default.
    phase = phase.resize((w, h), resample=Image.BICUBIC)
    # 'd' ('str') is a character code for a double-precision
    # floating-point number.
    arr_amplitude = np.array(amplitude, dtype='d')
    arr_phase = np.array(phase, dtype='d')
    amplitude.close()
    phase.close()
    arr_phase = math.pi * arr_phase / np.amax(arr_phase)
    obj = arr_amplitude * np.exp(1j * arr_phase)
    obj = np.absolute(obj)  # The Hi-Res complex image.
    if show:
        # Using PIL, the objects is shown as an image almost
        # completely white.
        # img_obj = Image.fromarray(obj)
        # img_obj.show()
        # Using matplotlib anc 'Greys_r' ('str') as value for 'cmap'
        # key [1], the object is shown like in Matlab R2009b.
        #
        # [1] unutbu. (2010). Display image as grayscale using matplotlib [Msg 1]. Message posted to
        # http://stackoverflow.com/questions/3823752/display-image-as-grayscale-using-matplotlib?answertab=votes#tab-top
        plt_img = plt.imshow(obj, cmap='Greys_r')
    return obj

#### 1.2. Generate the incident wave vectors.

In [ ]:
def gen_xy_led_ring(ledsperring, d):
    """Returns a partitioned list with the (x, y) coordinates of each
    LED of the LED ring.

    d ('int') is the distance in mm between LEDs and ledsperring
    ('int') is a list with the number of LEDs per ring from the
    center to the edge of this."""
    l_xy = list()
    l_row = list()
    l = list()
    for nleds in ledsperring:
        r = (d * nleds) / (2.0 * math.pi)
        theta_step = int(360 / nleds)
        range_theta = range(0, 360, theta_step)
        range_theta = range_theta[:nleds]
        for theta in range_theta:
            x = r * math.cos(math.radians(theta))
            y = r * math.sin(math.radians(theta))
            l_xy.append((x, y))
    order = math.ceil(math.sqrt(len(l_xy)))
    for i in range(order):
        for j in range(order):
            l_row.append((0.0, 0.0))
        l.append(l_row)
        l_row = list()
    len_lxy = len(l_xy)
    for i in range(order):
        for j in range(order):
            idx = i * (order + 1) + j
            if idx < len(l_xy):
                l[i][j] = l_xy[idx]
    return l

In [ ]:
def gen_xy_led_grid(n, d):
    """Returns a list with the (x, y) coordinates of each LED of a
    n-by-n LED grid.
    
    n ('int') is the number of LEDs per row or per column in the grid
    LED and d is the distance in mm between LEDs.
    """
    l_xy = list()
    x_max = math.floor((n / 2)) * d
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    l_range = list(range(x_min, x_max + 1, d))
    l_row = list()
    for i in l_range:
        for j in l_range:
            l_row.append((j, -i))  # (x, y)
        l_xy.append(l_row)
        l_row = list()
    return l_xy

In [ ]:
def generate_wave_vectors(l_xy, height):
    """Returns an array ('numpy.ndarray') with the magnitude of
    the x-component and y-component of the incident wave vectors that
    emerge from a LED illuminator. Each LED is represented by a 
    (x, y) tuple in l_xy ('list').
    
    height ('int') is the distance between the LED illuminator and
    the sample.
    """
    arr = np.array(l_xy)  # arr ('numpy.ndarray') contains (x, y)
                          # coordinates.
    arr = arr / height
    arr = np.arctan(arr)
    arr = np.sin(arr)
    arr = -arr
    return arr

In [ ]:
def round_half_up(num):
    """In Python 3, the round() function had changed. For example,
    round(2.5) returns 2 ('int') like round(1.5). It is possible to
    obtain 3 ('int') for 2.5 rounded using the decimal module [2].
    
    [2] Barthelemy. (2014). Python 3.x rounding behavior. Message
    posted to
    http://stackoverflow.com/questions/10825926/python-3-x-rounding-behavior
    """
    return int(decimal.Decimal(num).quantize(decimal.Decimal(1),
                                             rounding=decimal.ROUND_HALF_UP))

In [ ]:
def get_cft(wave_vectors, wavelen, ccdpx, zoom, na, w, h):
    """Returns the number of rows and columns of the Hi-Res output,
    vector waves, dkx ('float') and dky ('float'), and the coherent
    transfer function of the objective.
    
    wave_vectors is a numpy.ndarray and wavelen is a int. ccdpx
    ('float') is the sampling pixel size of the CCD, hirespx ('float') is the
    pixel size of the reconstruction, na ('float') is the numerical
    aperture of the lens, w ('int') the width in pixels of the output
    Hi-Res image and h ('int') its height. zoom ('int') is a STRANGE
    PARAMETER but it acts as it were a zoom.
    """
    k_0 = 2 * math.pi / wavelen
    hirespx = ccdpx / zoom
    dkx = 2 * math.pi / (hirespx * w)
    dky = 2 * math.pi / (hirespx * h)
    p = int(h / (ccdpx / hirespx))  # Number of rows of the output.
    q = int(w / (ccdpx / hirespx))  # Number of columns of the output.
    k = k_0 * wave_vectors;
    # From all rows, all columns, extract the first element.
    kx = k[:, :, 0]
    # Reshape xk ('numpy.ndarray') in 1-D array.
    kx = np.reshape(kx, len(kx.flat))
    ky = k[:, :, 1]
    ky = np.reshape(ky, len(ky.flat))
    cutoff_freq = na * k_0
    k_max = math.pi / ccdpx
    kxm, kym = np.meshgrid(np.arange(-k_max, k_max + 1,
                                     k_max / ((q - 1) / 2)),
                           np.arange(-k_max, k_max + 1,
                                     k_max / ((p - 1) / 2)))
    coherent_transfer_funct = ((kxm ** 2 + kym ** 2) < cutoff_freq ** 2)
    coherent_transfer_funct = coherent_transfer_funct.astype(float)
    return p, q, dkx, dky, kx, ky, coherent_transfer_funct

#### 3. Produce the ouput Lo-Res images.

In [ ]:
def num_str_zeros(num, n_digs, firstis1=False):
    """Returns a string that contains a sequence n-zeros followed by
    num ('int') as 'str', for example, num_str_zeros(89, 4) returns
    '0089'.
    """
    if firstis1:  # Begin the numeration in the filenames with 1.
        num += 1
    len_num = len(str(num))
    str_num = ''
    for i in range(n_digs - len_num):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
def find_out_dir(**kwargs):
    """Returns a string that contains a route of a directory.
    
    With replace ('bool') equal to False, this will create a new
    directory if dirname ('string') exists in <s3out>/<parentdir>/
    
    From blendjupyter.ipynb (Blender kernel)."""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    parentdir = kwargs.pop('parentdir', 'scanner')
    path = kwargs.pop('path', os.path.join(s3out, parentdir, dirname))
    replace = kwargs.pop('replace', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(find_out_dir.__name__, list(kwargs.keys())[-1]))
    if not replace:  # Then, create a new direcctory with a different
                     # name.
        while True:
            if not os.path.exists(path):
                print('Make directory:', path)
                os.makedirs(path)
                break
            else:
                str_num = path.split('-')[-1]
                num_dir = int(str_num)
                num_dir += 1
                str_num_dir = num_str_zeros(num_dir, len(str_num))
                path = '-'.join(path.split('-')[:-1]) + '-' + str_num_dir
    return path

In [ ]:
def generate_lores_set(obj, cft, leds, kx, ky, w, h, dkx, dky, p, q,
                       **kwargs):
    """Returns an array ('numpy.ndarray') with the sequence of Lo-Res
    images which are also stored.
    
    Each image only contains the amplitude information."""
    # Prefix in the name of each output file.
    prefix = kwargs.pop('prefix', 'lores-img_')
    # Extension of the output file.
    extension = kwargs.pop('extension', '.tif')
    outpath = kwargs.pop('outpath',
                         find_out_dir(dirname='lores-set-0001',
                                      parentdir='microscope'))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_lores_set.__name__, list(kwargs.keys())[-1]))
    obj_ft = np.fft.fftshift(np.fft.fft2(obj))
    lores_imgs_seq = list()  # The Lo-Res images sequence.
    for i in range(leds):
        kxc = round_half_up((w + 1) / 2.0 + kx[i] / dkx)
        kyc = round_half_up((h + 1) / 2.0 + ky[i] / dky)
        kyl = round_half_up(kyc - (p - 1) / 2.0);
        kyh = round_half_up(kyc + (p - 1) / 2.0);
        kxl = round_half_up(kxc - (q - 1) / 2.0);
        kxh = round_half_up(kxc + (q - 1) / 2.0);
        lores_imgs_seq_ft = (p / h) ** 2
        lores_imgs_seq_ft *= obj_ft[kyl - 1:kyh, kxl - 1:kxh]
        lores_imgs_seq_ft *= cft
        img_lores = np.absolute(np.fft.ifft2(np.fft.ifftshift(lores_imgs_seq_ft)))
        lores_imgs_seq.append(img_lores)
        str_i = num_str_zeros(i, len(str(leds)), True)
        filename = prefix + str_i + extension
        img_path = os.path.join(outpath, filename)
        # scipy.misc.imsave() rescales the dynamic range of the
        # pixel values [3]. Add cmin and cmax as parameters to
        # scipy.misc.toimage() as follow to prevent the rescaling.
        # Before, giving the np.uint8 to dtype('type') to img_lores
        # (numpy.ndarray) there was no effect by the rescaling.
        # 
        # [3] Lippens, S. (2016). Saving of images in scipy and
        # preventing dynamic range rescaling.
        scipy.misc.toimage(img_lores, cmin=0, cmax=255).save(img_path)
        print('Saved Lo-Res image:', img_path)
    return np.array(lores_imgs_seq)

In [ ]:
def simulate_set(**kwargs):
    """Returns the Lo-Res image sequence as a list of arrays
    ('numpy.ndarray') of the generate_lores_set() function.
    
    It calls all funtions in order to simulate the process of
    capturing photos in the Fourier Ptychography algorithm."""
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    ext = kwargs.pop('ext', '.tif')  # Extension of the output files.
    illuminator = kwargs.pop('illuminator', 'GRID')
    # For a LED illuminator type 'GRID', leds ('int') is the number of
    # LEDs per row or per column of a squared LED grid, e.g. 15;
    # instead, for a one type 'RING', leds is a list that contains the
    # number of LEDs by ring from the center to the edge of the LED
    # illuminator, e.g., [12, 18, 36].
    leds = kwargs.pop('leds', 15)
    d = kwargs.pop('d', 4)  # Distance in mm between LEDs.
    # Distance in mm between the LED grid and the sample.
    h = kwargs.pop('h', 90)
    # Does the first image have 1 at the end of its filename?
    namefrom1 = kwargs.pop('namefrom1', True)
    wavelen = kwargs.pop('wavelen', 0.63e-6)
    # Sampling pixel size of the CCD.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)
    # Pixel size of the reconstruction.
    hirespx = kwargs.pop('hirespx', ccdpx / 4)
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    # DANGER! IT COULD PIXELATE the output image.
    zoom = kwargs.pop('zoom', 4)
    amplitude = kwargs.pop('amplitude', os.path.join(s3path, 'img',
                                        'Alberto-Romero_Tornillos_flickr_1024x683.tif'))
    phase = kwargs.pop('phase', os.path.join(s3path, 'img',
                               'pabloaez_Valparaiso-054_flickr_4000x3000.png'))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(simulate_set.__name__, list(kwargs.keys())[-1]))
    l_xy_leds = list()
    if illuminator.lower() == 'grid' and isinstance(leds, int):
        l_xy_leds = gen_xy_led_grid(leds, d)
        leds = leds ** 2
    elif illuminator.lower() == 'ring' and isinstance(leds, list):
        l_xy_leds = gen_xy_led_ring(leds, d)
        leds = sum(leds)
    wvs = generate_wave_vectors(l_xy_leds, h)
    in_obj = generate_obj(amplitude, phase)
    hpx, wpx = in_obj.shape
    p, q, dkx, dky, kx, ky, cft = get_cft(wvs, wavelen, ccdpx, zoom,
                                          na, wpx, hpx)
    lores_imgs_seq = generate_lores_set(in_obj, cft, leds, kx, ky,
                                        wpx, hpx, dkx, dky, p, q,
                                        extension=ext)
    return lores_imgs_seq